In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

print("GPU disponible:", tf.config.list_physical_devices('GPU'))

GPU disponible: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [15]:
import tensorflow as tf
from tensorflow.keras import layers

# Tamaño y batch
img_size = (48, 48)
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

# Ruta del dataset
dataDir = "dataset/train"

# Carga de datos
val_ds = tf.keras.utils.image_dataset_from_directory(
    dataDir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    color_mode="grayscale",
    batch_size=batch_size
)

original_train_ds = tf.keras.utils.image_dataset_from_directory(
    dataDir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    color_mode="grayscale",
    batch_size=batch_size
)

# Clases
class_names = original_train_ds.class_names

# Normalización
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])

# Pipeline optimizado
train_ds = original_train_ds.cache().shuffle(1000).map(
    lambda x, y: (data_augmentation(normalization_layer(x), training=True), y),
    num_parallel_calls=AUTOTUNE
).prefetch(buffer_size=AUTOTUNE)

val_ds = val_ds.cache().map(
    lambda x, y: (normalization_layer(x), y),
    num_parallel_calls=AUTOTUNE
).prefetch(buffer_size=AUTOTUNE)

# Verifica GPU
print("Dispositivos GPU disponibles:", tf.config.list_physical_devices('GPU'))


Found 28709 files belonging to 6 classes.
Using 5741 files for validation.
Found 28709 files belonging to 6 classes.
Using 22968 files for training.
Dispositivos GPU disponibles: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Entrenamiento:

In [21]:
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

train_ds = train_ds.map(lambda x, y: (data_augmentation(x,training=True), y))

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])

model = models.Sequential([
    keras.layers.Input(shape=(48, 48, 1)),

    keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(len(class_names), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

early_stop = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True)


y_train = []
for _, labels in train_ds.unbatch():
    y_train.append(labels.numpy())
y_train = np.array(y_train)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights_dict = dict(enumerate(class_weights))

# Callback para detener entrenamiento si no mejora
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=6,
    restore_best_weights=True
)

# Callback para guardar el mejor modelo
checkpoint = ModelCheckpoint(
    "best_model.h5",          # nombre del archivo
    monitor='val_loss',       # métrica a vigilar
    save_best_only=True,      # solo guarda si mejora
    verbose=1
)

# Entrenamiento con callbacks
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=[early_stop, checkpoint],  # aquí incluyes ambos
    class_weight=class_weights_dict
)


Epoch 1/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.1854 - loss: 3.5659
Epoch 1: val_loss improved from inf to 1.88068, saving model to best_model.h5


718/718 ━━━━━━━━━━━━━━━━━━━━ 41s 45ms/step - accuracy: 0.1854 - loss: 3.5648 - val_accuracy: 0.2007 - val_loss: 1.8807
Epoch 2/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.1875 - loss: 1.8502
Epoch 2: val_loss improved from 1.88068 to 1.77453, saving model to best_model.h5


718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 38ms/step - accuracy: 0.1875 - loss: 1.8501 - val_accuracy: 0.2054 - val_loss: 1.7745
Epoch 3/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.1962 - loss: 1.7844
Epoch 3: val_loss did not improve from 1.77453
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.1962 - loss: 1.7844 - val_accuracy: 0.1949 - val_loss: 1.7958
Epoch 4/30
717/718 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.1892 - loss: 1.7732
Epoch 4: val_loss did not improve from 1.77453
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.1892 - loss: 1.7732 - val_accuracy: 0.1702 - val_loss: 1.8357
Epoch 5/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.1945 - loss: 1.7763
Epoch 5: val_loss did not improve from 1.77453
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.1945 - loss: 1.7763 - val_accuracy: 0.1825 - val_loss: 1.8078
Epoch 6/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.2114 - loss: 1.7640
Epoch 6: val_loss did not improve f

718/718 ━━━━━━━━━━━━━━━━━━━━ 41s 38ms/step - accuracy: 0.2556 - loss: 1.7165 - val_accuracy: 0.2564 - val_loss: 1.7287
Epoch 9/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.2769 - loss: 1.6935
Epoch 9: val_loss improved from 1.72870 to 1.63064, saving model to best_model.h5


718/718 ━━━━━━━━━━━━━━━━━━━━ 28s 38ms/step - accuracy: 0.2769 - loss: 1.6935 - val_accuracy: 0.3264 - val_loss: 1.6306
Epoch 10/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.2956 - loss: 1.6771
Epoch 10: val_loss did not improve from 1.63064
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.2956 - loss: 1.6771 - val_accuracy: 0.3268 - val_loss: 1.6487
Epoch 11/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.3095 - loss: 1.6607
Epoch 11: val_loss did not improve from 1.63064
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.3095 - loss: 1.6606 - val_accuracy: 0.3231 - val_loss: 1.7477
Epoch 12/30
717/718 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.3290 - loss: 1.6353
Epoch 12: val_loss did not improve from 1.63064
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.3290 - loss: 1.6353 - val_accuracy: 0.3268 - val_loss: 1.7753
Epoch 13/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.3408 - loss: 1.6248
Epoch 13: val_loss did not i

718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.3531 - loss: 1.5967 - val_accuracy: 0.3728 - val_loss: 1.5476
Epoch 15/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.3662 - loss: 1.5913
Epoch 15: val_loss improved from 1.54764 to 1.48807, saving model to best_model.h5


718/718 ━━━━━━━━━━━━━━━━━━━━ 42s 38ms/step - accuracy: 0.3662 - loss: 1.5913 - val_accuracy: 0.4067 - val_loss: 1.4881
Epoch 16/30
717/718 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.3689 - loss: 1.5741
Epoch 16: val_loss did not improve from 1.48807
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.3689 - loss: 1.5741 - val_accuracy: 0.4041 - val_loss: 1.5110
Epoch 17/30
717/718 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.3918 - loss: 1.5585
Epoch 17: val_loss did not improve from 1.48807
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.3919 - loss: 1.5585 - val_accuracy: 0.3705 - val_loss: 1.5866
Epoch 18/30
717/718 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.4018 - loss: 1.5347
Epoch 18: val_loss did not improve from 1.48807
718/718 ━━━━━━━━━━━━━━━━━━━━ 41s 37ms/step - accuracy: 0.4017 - loss: 1.5347 - val_accuracy: 0.3520 - val_loss: 1.7387
Epoch 19/30
717/718 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.4047 - loss: 1.5295
Epoch 19: val_loss improved 

718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.4047 - loss: 1.5295 - val_accuracy: 0.4241 - val_loss: 1.4756
Epoch 20/30
717/718 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.4105 - loss: 1.5139
Epoch 20: val_loss improved from 1.47561 to 1.43271, saving model to best_model.h5


718/718 ━━━━━━━━━━━━━━━━━━━━ 28s 38ms/step - accuracy: 0.4105 - loss: 1.5139 - val_accuracy: 0.4440 - val_loss: 1.4327
Epoch 21/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.4198 - loss: 1.5106
Epoch 21: val_loss did not improve from 1.43271
718/718 ━━━━━━━━━━━━━━━━━━━━ 28s 39ms/step - accuracy: 0.4198 - loss: 1.5106 - val_accuracy: 0.4149 - val_loss: 1.4944
Epoch 22/30
717/718 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.4228 - loss: 1.5041
Epoch 22: val_loss did not improve from 1.43271
718/718 ━━━━━━━━━━━━━━━━━━━━ 40s 37ms/step - accuracy: 0.4228 - loss: 1.5041 - val_accuracy: 0.3383 - val_loss: 1.7308
Epoch 23/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.4258 - loss: 1.4892
Epoch 23: val_loss did not improve from 1.43271
718/718 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.4258 - loss: 1.4892 - val_accuracy: 0.3701 - val_loss: 1.6144
Epoch 24/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.4292 - loss: 1.4842
Epoch 24: val_loss did not i

Descargar el modelo:

In [23]:
from google.colab import files
files.download("best_model.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Matriz de confusion:

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Predicciones
y_pred = np.argmax(model.predict(val_ds), axis=1)

# Etiquetas verdaderas
y_true = []
for _, labels in val_ds.unbatch():
    y_true.append(labels.numpy())
y_true = np.array(y_true)

# Reporte
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=class_names, zero_division=0))



In [19]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Matriz de Confusión')
plt.show()


NameError: name 'y_true' is not defined

## Evaluacion del accuaracy

In [22]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/test",
    image_size=img_size,
    color_mode="grayscale",
    batch_size=batch_size
)

test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y)).cache().prefetch(buffer_size=AUTOTUNE)


test_loss, test_acc = model.evaluate(test_ds)
print(f"Test accuracy: {test_acc:.2f}")


Found 7178 files belonging to 6 classes.
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.4379 - loss: 1.4423
Test accuracy: 0.44


## Grafica para ver la accuaracy del modelo:


In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Train Accuracy')
plt.plot(epochs_range, val_acc, label='Val Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Val Loss')
plt.legend()
plt.title('Loss')
plt.show()


## Grafico para ver el desbalance:

In [ ]:
import matplotlib.pyplot as plt

class_counts = {'angry': 3995, 'disgust': 436, 'fear': 4097, 'happy': 7215, 'neutral': 4965, 'sad': 4830, 'surprise': 3171}

plt.figure(figsize=(10, 6))
plt.bar(class_counts.keys(), class_counts.values(), color='skyblue')
plt.title("Distribución de clases en el dataset de entrenamiento")
plt.xlabel("Emoción")
plt.ylabel("Cantidad de imágenes")
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()


## Guardar el modelo

In [ ]:
model.save("emotion_model.h5")
# model.save("emotion_model.keras") <-- Esta forma la recomienda keras ya que el que usamos se ve mas antiguo, pero ocupo confirmar con la documentación